In [ ]:
from bs4 import BeautifulSoup
from math import log
import pandas as pd

In [ ]:
def init_score(guess):
    for i, letter in enumerate(guess):
        if letter == solution[i]:
            guess_scores[i] = g
            print(g)
        elif letter in solution:
            guess_scores[i] = y
            print(y)
        else:
            misses.append(letter)
            guess_scores[i] = m
            print(m)
    return guess_scores, misses
# print(misses)
# print(guess_scores)

In [ ]:
def remove_words(options, misses, guess, guess_scores):
    removes = []
    print(f"length of options before anything: {len(options)}\n")

    # removing any words with letters with from the misses list

    for word in options:
        for letter in misses:
            if letter in word:
                # print(word, letter)
                removes.append(word)
                break

    options_2 = [x for x in options if x not in removes]
    print(f"length of options after misses: {len(options_2)}\n")

    # removing any words with letters that don't match the green letter at its placement

    for word in options_2:
        for i, score in enumerate(guess_scores):
            if score == 2:
                true_letter = guess[i]
                if word[i] != true_letter:
                    # print(word, guess, true_letter)
                    removes.append(word)
                    break

    options_3 = [x for x in options_2 if x not in removes]
    print(f"length of options after misses and green letter: {len(options_3)}\n")

    # removing any words that don't contain the yellow letters

    yellow_letters = []
    yl = {}                                
    for i, digit in enumerate(guess_scores):
        if digit == 1:
            if guess[i] in yellow_letters:
                yl[guess[i]].append(i)
            else:
                yl[guess[i]] = [i]
            yellow_letters.append(guess[i])
    
    for word in options_3:
        for letter in yellow_letters:
            if letter not in word:
                removes.append(word)
                break

    options_4 = [x for x in options_3 if x not in removes]

    print(f"length of options after misses, green letter, and yellow letter: {len(options_4)}\n")

    for word in options_4: # TODO: THIS IS ANOTHER AREA THAT NEEDS WORK
        for letter in yl:
            for index in yl[letter]:    
                if word[index] == letter:
                    removes.append(word)
                    break

    options_5 = [x for x in options_4 if x not in removes]

    print(f"length of options after misses, green letter, yellow letter, and yellow letter placement elimination: {len(options_5)}\n")

    for word in options_5:
        for letter in yl:
            if word.count(letter) < len(yl[letter]):
                removes.append(word)
                break
            
    options_6 = [x for x in options_5 if x not in removes]

    ml = []
    multiple_letters = {False: ''}

    for letter in yl:
        if len(yl[letter]) > 1:
            val = {letter: len(yl[letter])}
            ml.append(val)
    for letter in yl:
        if len(yl[letter]) > 1:
            multiple_letters = {True: ml}
            break
        else:
            multiple_letters = {False: ''}
    
    print(f"length of options after misses, green letter, yellow letter, yellow letter placement, and yellow letter count: {len(options_6)}\n")

    return options_6, multiple_letters


In [ ]:
def filter_by_score(options, guess, guess_scores):
    scores = {}
    for word in options:
        score = 0
        for i, letter in enumerate(word):
            if guess_scores[i] == 0:
                continue
            elif guess_scores[i] == 1:
                if guess[i] in word:
                    score += guess_scores[i]
            elif guess_scores[i] == 2:
                if letter == guess[i]:
                    score += guess_scores[i]
        scores[word] = score

    best_score = max(scores.values())
    return_options = []
    for word in options:
        if scores[word] == best_score:
            return_options.append(word)
    # print(return_options)
    return return_options
        

In [ ]:
def get_letter_freq():

    s = '''Letter	Count	 	Letter	Frequency
    E	21912	 	E	12.02
    T	16587	 	T	9.10
    A	14810	 	A	8.12
    O	14003	 	O	7.68
    I	13318	 	I	7.31
    N	12666	 	N	6.95
    S	11450	 	S	6.28
    R	10977	 	R	6.02
    H	10795	 	H	5.92
    D	7874	 	D	4.32
    L	7253	 	L	3.98
    U	5246	 	U	2.88
    C	4943	 	C	2.71
    M	4761	 	M	2.61
    F	4200	 	F	2.30
    Y	3853	 	Y	2.11
    W	3819	 	W	2.09
    G	3693	 	G	2.03
    P	3316	 	P	1.82
    B	2715	 	B	1.49
    V	2019	 	V	1.11
    K	1257	 	K	0.69
    X	315	 	X	0.17
    Q	205	 	Q	0.11
    J	188	 	J	0.10
    Z	128	 	Z	0.07
    '''

    raw_df = pd.DataFrame([x.split('\t') for x in s.split('\n') if x != ''], columns=['letter', 'count', '','letter2', 'frequency'])
    raw_df.drop(columns=['', 'count','letter2'], inplace=True)
    raw_df.drop(index=0, inplace=True)
    df = raw_df.set_index('letter')
    d = df.to_dict()['frequency']
    keys = [x.strip().lower() for x in d]
    return {key:float(f) for (key,f) in zip(keys, d.values()) if key !=''}

get_letter_freq()

In [ ]:
# calculate confidence based strictly on how many green and yellow letters there are

def green_yellow_confidence(green_count, yellow_count, pool_size, total_letters=5, weight_green=.7, weight_yellow=.3):
    # Avoid division by zero
    if pool_size == 0:
        return 0

    # TODO: add a factor for the remaining words in the alphabet
    # TODO: add a factor for the popularity of the letter (z should be lower in rank than s)

    score = ((green_count * weight_green + yellow_count * weight_yellow) / total_letters) * (1 / pool_size)
    return score

print(green_yellow_confidence(1, 0, 10))
print(green_yellow_confidence(0, 5, 10))


In [ ]:
def build_confidence_dict(guess, words, guess_scores, multiple_letters):
    word_info = []
    
    # counting the amount of green and yellow letters in the guess
    gl = [x for i,x in enumerate(guess) if guess_scores[i] == 2]
    yl = [x for i,x in enumerate(guess) if guess_scores[i] == 1]
    
    # building a dictionary of the remaining guesses and 
    # how many green and yellow letters they have in common
    # with the guess
    
    for word in words:
        s_word = list(set(word))    # TODO: right now we are assuming that there are no double letters
        yellow_letters = 0
        green_letters = 0
        for i, letter in enumerate(s_word):
            if letter in gl:
                green_letters += 1
            elif letter in yl:
                yellow_letters += 1
        # print(word, green_letters, yellow_letters)
        word_info.append({word: {'green': green_letters, 'yellow': yellow_letters}})

    confidence_dict = {}
    for d in word_info:
        for word in d:
            confidence_dict[word] = green_yellow_confidence(d[word]['green'], d[word]['yellow'], len(words))
    
    letter_freq = get_letter_freq()

    # compiling a dictionary of each word and the sum of frequency score of each of its letters

    freq_scores = {}
    for word in words:
        freq_scores[word] = 0
        for letter in word:
            freq_scores[word] += letter_freq[letter]

    # adding the frequency score to the confidence score

    for word in confidence_dict:
        confidence_dict[word] += (freq_scores[word] * 0.01)

    # adding a penalty for words with multiple letters not accounted for

    if not list(multiple_letters.keys())[0]:
        for word in confidence_dict:
            if len(set(word)) < len(word):
                confidence_dict[word] -= (0.15 * (len(word)-len(set(word))))
    else:
        for word in confidence_dict:
            for letter in word:
                if word.count(letter) > 1:
                    if letter not in list(multiple_letters[True][0].keys()):
                        confidence_dict[word] -= 0.15

    for key in confidence_dict:
        confidence_dict[key] = round(confidence_dict[key], 4)
    
    return dict(sorted(confidence_dict.items(), key=lambda item: item[1], reverse=True))
    


    

# build_confidence_dict('serai', ['young','would', 'known', 'blood', 'block', 'fully', 'touch', 'funny', 'buddy', 'mouth', 'bound'], [0, 0, 0, 0, 0], {False: ''})

In [ ]:
# response = get('https://www.wordunscrambler.net/word-list/wordle-word-list')
# bs = BeautifulSoup(response.text, 'html.parser')
# all_a_html = bs.find_all('a')
# all_a = [a.text for a in all_a_html]
# all_words = [word for word in all_a if len(word) == 5]
# pool = [word for word in all_words if word not in past_words]

In [ ]:
# download the text files so it looks purty

# we need to adjust ALL the weights to not be arbitrary

# make a threshold for confidence scores (if it's lower than x, guess a word that will give lots of information)

# make an information gain calculator

# Jocelyn guessed eaten when it was taken and we got an error]\

# input that a second e gave me a 0 score and it messed it up

# guessed state and it removed options with t because the second t was a 0 score

In [ ]:
def compare_lists(popularity, confidence):
    weight_commonality = 0.55
    weight_confidence = 0.45
    combined_scores = {}
    for i, word in enumerate(popularity):
        rank = i + 1  # Rank starting from 1
        normalized_rank = 1 / rank  # Normalizing rank

        # Combined score calculation
        combined_score = (normalized_rank * weight_commonality) + (confidence[word] * weight_confidence)
        combined_scores[word] = round(combined_score,4)

    # Sort words by combined score
    best_guesses = sorted(combined_scores, key=combined_scores.get, reverse=True)
    return dict(sorted(combined_scores.items(), key=lambda item: item[1], reverse=True))

compare_lists(['reply'],{'reply': 0.4829})

In [ ]:
all_options = pd.read_csv('options.csv')['word'].tolist()
keep_guessing = True
count = 0
correct_word = ''

while keep_guessing:
    count += 1
    guess = input('What did we guess?')
    if guess == '':
        break
    misses = input('What letters did we miss?')
    guess_scores = input('What were the scores?')
    misses = [x for x in misses]
    guess_scores = [int(x) for x in guess_scores]

    print(f'\nguess: {guess}\n')
    print(f'misses: {misses}\n')
    print(f'guess_scores: {guess_scores}\n')

    o, ml = remove_words(options, misses, guess, guess_scores)
    # print(f'options: {options}\n')
    # final_options = filter_by_score(o, guess, guess_scores) # is this actually necessary?
    d = build_confidence_dict(guess, o, guess_scores, ml)
    print(f'\n\nfinal option(s) ranked by confidence level (max 10):  {list(d.items())[:10]}\n')
    print(f'top final option(s) ranked by word popularity (max 10): {o[:10]}\n')
    print(f'final option(s) ranked by confidence * popularity (max 10): {list(compare_lists(o, d).items())[:10]}\n')



    if input('Take a look at the options below and input your guess. If we guessed correctly, type "correct", otherwise, hit enter') == 'correct':
        count +=1
        keep_guessing = False
        correct_word = input(f'\nNice job! We got it in {count} guesses! What was the correct word?\n')
        continue
    
    options = o
    # print(len(options), len(all_options))

if correct_word in all_options:
    all_options.remove(correct_word)
df = pd.DataFrame(all_options, columns=['word'])
df.to_csv('options.csv', index=False)

This idea can be expressed mathematically. The probability (p) of finding a word with a certain property (such as the letter A) can be calculated by dividing the total number of words containing A (represented as MA) by the number of all words (M). So p = MA / M. At the same time, the information (I), meaning “The word contains an A,” reduces the space of all possibilities (M) by the factor ½I. We can present that as MA = ½I x M.

By inserting both equations into each other, one can conclude with a formula that combines information content and probability: p = ½I x M / M, so p = ½I. This can also be reversed and solved for I: I = –log2p.

In [ ]:
def info_gain(words, hint):
    count = 0
    for word in words:
        if hint not in word:
            count += 1
    print(len(words)-count, len(words))
    print(f"bits necessary to get the correct answer: {log(1/(1/len(words)), 2)}")
    print(f"information gained from removing words containing {hint}: {log(1/((len(words)-count)/len(words)), 2)}")

s = 'AD	BE	IF	ME AM	HI	ON	MY AS	GO	IS	NO AT	TV	IT	TO'
s = s.lower().split()
info_gain(s, 'a')